In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cd '/content/drive/MyDrive'

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/spam.csv')

In [ ]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [ ]:
df.isnull().sum()

,0
Category,0
Message,0


In [ ]:
df['Category'].value_counts()

,count
Category,
ham,4825
spam,747


In [ ]:
print(df['Category'].value_counts()/len(df))

Category
ham     0.865937
spam    0.134063
Name: count, dtype: float64


In [ ]:
df['label'] = [1 if x =='spam' else 0 for x in df['Category']]

In [ ]:
df.head()

,Category,Message,label
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [ ]:
pip install tensorflow==2.9.0 tensorflow-hub==0.12.0 tensorflow-text


ERROR: Operation cancelled by user


In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
df_spam = df[df['label'] == 1]

In [ ]:
df_spam.info()

<class 'pandas.core.frame.DataFrame'>
Index: 747 entries, 2 to 5567
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  747 non-null    object
 1   Message   747 non-null    object
 2   label     747 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 23.3+ KB


In [ ]:
df_ham = df[df['label']==0]

In [ ]:
df_ham.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4825 entries, 0 to 5571
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  4825 non-null   object
 1   Message   4825 non-null   object
 2   label     4825 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 150.8+ KB


In [ ]:
df_down_sample = df_ham.sample(len(df_spam))

In [ ]:
df_balanced = pd.concat([df_down_sample, df_spam])

In [ ]:
df_balanced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1494 entries, 4714 to 5567
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  1494 non-null   object
 1   Message   1494 non-null   object
 2   label     1494 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 46.7+ KB


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_balanced['Message'],df_balanced['label'], test_size=.3,random_state=42)

In [ ]:
x_train.shape, x_test.shape

((1045,), (449,))

In [ ]:
y_train.shape, y_test.shape

((1045,), (449,))

In [ ]:
bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

preprocessed_text = bert_preprocess(text_input)
bert_output = bert_encoder(preprocessed_text)
pooled_output = bert_output['pooled_output']
dropout = tf.keras.layers.Dropout(0.1, name='dropout')(pooled_output)
classifier = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(dropout)
model = tf.keras.Model(inputs=[text_input], outputs=[classifier])


In [ ]:
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [ ]:
METRICS  = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(x_train, y_train, epochs=1,validation_data=(x_test, y_test))

33/33 [==============================] - 782s 24s/step - loss: 0.6414 - accuracy: 0.6268 - precision: 0.6233 - recall: 0.6136 - val_loss: 0.5767 - val_accuracy: 0.7305 - val_precision: 0.9440 - val_recall: 0.5086


In [ ]:
pred = model.predict(x_test)

 8/15 [===============>..............] - ETA: 1:49

In [ ]:
pred = [1 if x >=0.5 else 0 for x in pred]

In [ ]:
pred[:10]

In [ ]:
pred = pd.DataFrame(pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

In [ ]:
pip install transformers

In [ ]:
   x_train = [i for i in x_train]

In [ ]:
x_test = [i for i in x_test]

In [ ]:
import tensorflow as tf
import numpy as np
from transformers import BertTokenizer, TFBertForSequenceClassification

# Load the pre-trained BERT tokenizer and model
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = TFBertForSequenceClassification.from_pretrained(model_name)


max_length = 128  # Adjust as needed
X_train_tokens = tokenizer(x_train, padding='max_length', truncation=True, max_length=max_length, return_tensors="tf")

# Get input_ids and attention_mask
input_ids = X_train_tokens["input_ids"]
attention_mask = X_train_tokens["attention_mask"]

# Define the input layers
input_ids_input = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
attention_mask_input = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

# BERT encoder
output = bert_model(input_ids=input_ids_input, attention_mask=attention_mask_input)

# Get the pooled output
pooled_output = output.logits

# Dropout layer
dropout_output = tf.keras.layers.Dropout(0.1, name='dropout')(pooled_output)

# Output layer for binary classification (spam or non-spam)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(dropout_output)

# Define the model
model = tf.keras.Model(inputs=[input_ids_input, attention_mask_input], outputs=[output_layer])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Display a summary of the model architecture
model.summary()

# Convert y_train to a numpy array
y_train = np.array(y_train)

# Train the model
model.fit([input_ids, attention_mask], y_train, epochs=3, batch_size=32)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_for_sequence_class  TFSequenceClassifierOutput   1094837   ['input_ids[0][0]',           
 ification (TFBertForSequen  (loss=None, logits=(None,    78         'attention_mask[0][0]']      
 ceClassification)           2),                                                            

In [ ]:

# Tokenize the evaluation text and pad sequences
max_length = 128  # Adjust as needed
X_eval_tokens = tokenizer(x_test, padding='max_length', truncation=True, max_length=max_length, return_tensors="tf")

# Get input_ids and attention_mask for evaluation
input_ids_eval = X_eval_tokens["input_ids"]
attention_mask_eval = X_eval_tokens["attention_mask"]

# Evaluate the model on the evaluation dataset
evaluation_results = model.evaluate([input_ids_eval, attention_mask_eval], y_test)

# Print evaluation results
print("Loss:", evaluation_results[0])
print("Accuracy:", evaluation_results[1])


15/15 [==============================] - 9s 270ms/step - loss: 0.0799 - accuracy: 0.9800
Loss: 0.0798691064119339
Accuracy: 0.9799554347991943


In [ ]:
predictions = model.predict([input_ids_eval, attention_mask_eval])

# The predictions will be in the form of probabilities (floats)
# You can threshold the probabilities to make binary predictions
threshold = 0.5  # Adjust as needed
binary_predictions = [1 if p >= threshold else 0 for p in predictions]

# Print the binary predictions
print("Binary Predictions:", binary_predictions)

15/15 [==============================] - 10s 271ms/step
Binary Predictions: [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, binary_predictions))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       217
           1       0.97      0.99      0.98       232

    accuracy                           0.98       449
   macro avg       0.98      0.98      0.98       449
weighted avg       0.98      0.98      0.98       449

